In [0]:
# Databricks notebook source - 02_PDF_to_Delta_ETL
# Project: India Air Quality Batch ETL Pipeline
# Step 2: Extract tables from PDF → Clean DataFrame → Workspace Files (CSV + Parquet)

# Install PDF table extraction libraries
%pip install tabula-py pdfplumber pandas pyarrow

import tabula
import pandas as pd
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

print("🚀 Starting PDF to Workspace Files ETL Pipeline...")

# Step 1: Auto-detect LATEST PDF
pdf_dir = "files/aqi/raw"
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

latest_pdf = None
latest_ctime = -1
for f in pdf_files:
    ctime = os.path.getctime(os.path.join(pdf_dir, f))
    if ctime > latest_ctime:
        latest_pdf = f
        latest_ctime = ctime

pdf_path = os.path.join(pdf_dir, latest_pdf)
print(f"📄 Processing: {latest_pdf}")

# Extract date from filename
yyyymmdd = latest_pdf.split('_')[2].split('.')[0]
report_date = datetime.strptime(yyyymmdd, "%Y%m%d").date()
print(f"📅 Report date: {report_date}")

# Step 2: Extract ALL tables
dfs = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, pandas_options={'header': None})
print(f"📊 Found {len(dfs)} tables")

# Step 3: Filter AQI tables
relevant_dfs = []
for i, df in enumerate(dfs):
    if df.shape[0] < 3 or df.shape[1] < 6:
        continue
    sample_text = ' '.join(df.iloc[0:3].astype(str).values.flatten()).lower()
    if any(keyword in sample_text for keyword in ['s.no', 'city', 'poor', 'moderate', 'satisfactory']):
        print(f"✅ Table {i+1} is AQI data: {df.shape}")
        relevant_dfs.append(df)

# Step 4: Concatenate tables
aqi_df = pd.concat(relevant_dfs, ignore_index=True)
print(f"📈 Combined shape: {aqi_df.shape}")

# Step 5: Clean CPCB columns
print("🧹 Standardizing columns...")
aqi_df = aqi_df.iloc[1:].reset_index(drop=True)
aqi_df = aqi_df.dropna(how='all')

column_mapping = {0: 'SNo', 1: 'City', 2: 'Category', 3: 'AQI_Value', 4: 'Dominant_Pollutant', 5: 'Stations'}

if aqi_df.shape[1] == 6:
    aqi_df.columns = [column_mapping.get(i, f'Extra_{i}') for i in range(6)]
elif aqi_df.shape[1] > 6:
    aqi_df = aqi_df.iloc[:, :6]
    aqi_df.columns = [column_mapping.get(i, f'Extra_{i}') for i in range(6)]

# Clean data
for col in ['SNo', 'City', 'Category', 'Dominant_Pollutant', 'Stations']:
    if col in aqi_df.columns:
        aqi_df[col] = aqi_df[col].astype(str).str.strip()
aqi_df['AQI_Value'] = pd.to_numeric(aqi_df['AQI_Value'], errors='coerce')
aqi_df['report_date'] = str(report_date)
aqi_df['ingestion_timestamp'] = pd.Timestamp.now()

# Filter valid rows
aqi_df = aqi_df[
    (aqi_df['City'].str.len() > 1) &
    (aqi_df['AQI_Value'].notna()) &
    (~aqi_df['City'].str.contains('Page|absent|Note|Health', case=False, na=False)) &
    (~aqi_df['SNo'].str.contains('nan', na=False))
].reset_index(drop=True)

print(f"✅ Cleaned data: {aqi_df.shape} rows")
display(aqi_df.head(10))

# Step 6: Save to YOUR GitHub-synced Workspace Files (CSV + Parquet)
delta_dir = "files/aqi/delta"
csv_path = f"{delta_dir}/aqi_daily_all.csv"
parquet_path = f"{delta_dir}/aqi_daily_all.parquet"

# Create directory
os.makedirs(delta_dir, exist_ok=True)

# Read existing data (if exists) and append today's data
try:
    existing_df = pd.read_csv(csv_path)
    print(f"📂 Found existing data: {len(existing_df)} rows")
    combined_df = pd.concat([existing_df, aqi_df], ignore_index=True)
except:
    print("📂 No existing data, creating new")
    combined_df = aqi_df

# Save both formats
combined_df.to_csv(csv_path, index=False)
combined_df.to_parquet(parquet_path, index=False)

print(f"✅ Saved {len(combined_df)} total records:")
print(f"   📄 CSV: {csv_path}")
print(f"   🪶 Parquet: {parquet_path}")

# Step 7: Create Spark DataFrame for analysis
spark_df = spark.createDataFrame(combined_df)
spark_df.createOrReplaceTempView("aqi_daily")

# Step 8: Verify TOP polluted cities
print("\n🏭 TOP 10 MOST POLLUTED:")
display(spark.sql("""
    SELECT * FROM aqi_daily 
    ORDER BY AQI_Value DESC 
    LIMIT 10
"""))

print("\n🎉 ETL COMPLETE!")
print(f"📊 Query anytime: SELECT * FROM aqi_daily")
print(f"📂 GitHub syncs: files/aqi/delta/aqi_daily_all.csv")
